# --- Day 7: Camel Cards ---
Your all-expenses-paid trip turns out to be a one-way, five-minute ride in an airship. (At least it's a cool airship!) It drops you off at the edge of a vast desert and descends back to Island Island.

"Did you bring the parts?"

You turn around to see an Elf completely covered in white clothing, wearing goggles, and riding a large camel.

"Did you bring the parts?" she asks again, louder this time. You aren't sure what parts she's looking for; you're here to figure out why the sand stopped.

"The parts! For the sand, yes! Come with me; I will show you." She beckons you onto the camel.

After riding a bit across the sands of Desert Island, you can see what look like very large rocks covering half of the horizon. The Elf explains that the rocks are all along the part of Desert Island that is directly above Island Island, making it hard to even get there. Normally, they use big machines to move the rocks and filter the sand, but the machines have broken down because Desert Island recently stopped receiving the parts they need to fix the machines.

You've already assumed it'll be your job to figure out why the parts stopped when she asks if you can help. You agree automatically.

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

Five of a kind, where all five cards have the same label: AAAAA
Four of a kind, where four cards have the same label and one card has a different label: AA8AA
Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
High card, where all cards' labels are distinct: 23456
Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:
```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.
T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.
Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?

## --- Part Two ---
To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:
```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
KK677 is now the only two pair, making it the second-weakest hand.
T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.
With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [1]:
from pathlib import Path
import os

yr = 2023
d = 7

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  hands = []
  bids = []
  for l in inp.split('\n'):
    hands.append(l.split(' ')[0])
    bids.append(int(l.split(' ')[1]))
  return {'hands': hands, 'bids': bids}

In [3]:
def hand_to_counts(h):
  seen_cards = []
  counts = []
  for c in h:
    if c not in seen_cards:
      seen_cards.append(c)
      counts.append(1)
    else:
      ci = seen_cards.index(c)
      counts[ci] += 1
  return list(sorted(counts, reverse=True))

def is_5_of_kind(h):
  cnts = hand_to_counts(h)
  return cnts[0]==5

def is_4_of_kind(h):
  cnts = hand_to_counts(h)
  return cnts[0]==4

def is_full_house(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==3 and hand_to_counts(h)[1]==2

def is_3_of_kind(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==3 and hand_to_counts(h)[1]!=2

def is_2_pair(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==2 and hand_to_counts(h)[1]==2

def is_1_pair(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==2 and hand_to_counts(h)[1]!=2

def is_high_card(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==1

def generate_hand_to_priority(jokers_wild=False):
  from itertools import product
  from copy import deepcopy
  from functools import cmp_to_key
  from tqdm.notebook import tqdm

  def compare_hands_by_lead_cards(h1, h2):
    if not jokers_wild:
      card_order = {
                    'A':12,
                    'K':11,
                    'Q':10,
                    'J':9,
                    'T':8,
                    '9':7,
                    '8':6,
                    '7':5,
                    '6':4,
                    '5':3,
                    '4':2,
                    '3':1,
                    '2':0
                  }
    else:
      card_order = {
              'A':12,
              'K':11,
              'Q':10,
              'T':9,
              '9':8,
              '8':7,
              '7':6,
              '6':5,
              '5':4,
              '4':3,
              '3':2,
              '2':1,
              'J':0,
            }
    for i in range(5):
      if card_order[h1[i]] < card_order[h2[i]]:
        return -1
      elif card_order[h1[i]] > card_order[h2[i]]:
        return 1
    return 0

  def get_highest_priority_hand_type(hand):


    prio = {'4_of_kind': 5,
            'full_house': 4,
            '3_of_kind': 3,
            '2_pair': 2,
            '1_pair': 1,
            'high_card': 0}

    hand = list(hand)
    hands_to_check = [hand]

    # If the jokers are wild then we will check all possible hands
    # that can be generated from replacing the jokers in the hand
    if jokers_wild:
      joker_locs = [i for i, c in enumerate(hand) if c=='J']
      potential_new_jokers = product('AKQJT98765432', repeat=len(joker_locs))
      for pnj in potential_new_jokers:
        new_hand = deepcopy(hand)
        for i, jl in enumerate(joker_locs):
          new_hand[jl] = pnj[i]
        hands_to_check.append(new_hand)


    hand_types = []
    for h in hands_to_check:
      if is_5_of_kind(h):
        return '5_of_kind'
      elif is_4_of_kind(h):
        hand_types.append('4_of_kind')
      elif is_full_house(h):
        hand_types.append('full_house')
      elif is_3_of_kind(h):
        hand_types.append('3_of_kind')
      elif is_2_pair(h):
        hand_types.append('2_pair')
      elif is_1_pair(h):
        hand_types.append('1_pair')
      elif is_high_card(h):
        hand_types.append('high_card')
      else:
        raise Exception(h, " COULDN'T BE CLASSIFIED")

    return max(hand_types, key=lambda x: prio[x])


  # We will build lists of all possible hands,
  # sort them by lead cards, and then use them to determine
  # overall priority for every possible hand
  five_of_kinds = []
  four_of_kinds = []
  full_houses = []
  three_of_kinds = []
  two_pairs = []
  one_pairs = []
  high_cards = []


  possible_combos = product('AKQJT98765432', repeat=5)
  for pp in tqdm(list(possible_combos)):
    hand_type = get_highest_priority_hand_type(pp)
    if hand_type == '5_of_kind':
      five_of_kinds.append(pp)
    elif hand_type == '4_of_kind':
      four_of_kinds.append(pp)
    elif hand_type == 'full_house':
      full_houses.append(pp)
    elif hand_type == '3_of_kind':
      three_of_kinds.append(pp)
    elif hand_type == '2_pair':
      two_pairs.append(pp)
    elif hand_type == '1_pair':
      one_pairs.append(pp)
    elif hand_type == 'high_card':
      high_cards.append(pp)



  # Once we know which hands belong to which category, we just need
  # to sort each category by the lead cards
  five_of_kinds = sorted(five_of_kinds,
                         key=cmp_to_key(compare_hands_by_lead_cards))
  four_of_kinds = sorted(four_of_kinds,
                         key=cmp_to_key(compare_hands_by_lead_cards))
  full_houses = sorted(full_houses,
                       key=cmp_to_key(compare_hands_by_lead_cards))
  three_of_kinds = sorted(three_of_kinds,
                          key=cmp_to_key(compare_hands_by_lead_cards))
  two_pairs = sorted(two_pairs,
                     key=cmp_to_key(compare_hands_by_lead_cards))
  one_pairs = sorted(one_pairs,
                     key=cmp_to_key(compare_hands_by_lead_cards))
  high_cards = sorted(high_cards,
                      key=cmp_to_key(compare_hands_by_lead_cards))

  h2p = {}
  for i, h in enumerate(high_cards + one_pairs + two_pairs + three_of_kinds +
                        full_houses + four_of_kinds + five_of_kinds):
    h2p[''.join(h)] = i

  return h2p

In [4]:
def total_winnings(formatted_input, hand_to_priority):
  hb = list(zip(formatted_input['hands'], formatted_input['bids']))
  hb.sort(key=lambda x: hand_to_priority[x[0]])
  return sum([x[1]*(i+1) for i, x in enumerate(hb)])

### A faster way to do this would be to just generate the hand_to_priorities maps only for the hands that we have in the input.

##### I felt like doing it this way though so that any hypothetical future hands could be easily looked up

In [5]:
import time

t = time.time()

formatted_input = format_input(inp)

h2p = generate_hand_to_priority()
h2pjw = generate_hand_to_priority(jokers_wild=True)

tt = time.time()

print(total_winnings(formatted_input, h2p))
print(total_winnings(formatted_input, h2pjw))


print("\nRUNTIME (GENERATE ALL POSSIBLE HAND PRIORITIES): ", tt-t)
print("RUNTIME (CALCULATE WINNINGS): ", time.time()-tt)
print("RUNTIME (TOTAL): ", time.time()-t)

  0%|          | 0/371293 [00:00<?, ?it/s]

  0%|          | 0/371293 [00:00<?, ?it/s]

250602641
251037509

RUNTIME (GENERATE ALL POSSIBLE HAND PRIORITIES):  136.85615253448486
RUNTIME (CALCULATE WINNINGS):  0.001994609832763672
RUNTIME (TOTAL):  136.85814714431763
